In [1]:
! pip install openai

     -------------------------------------- 70.1/70.1 kB 950.1 kB/s eta 0:00:00
     -------------------------------------- 319.8/319.8 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 56.1/56.1 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 60.0/60.0 kB 1.6 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
! pip install datasets


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 468.7/468.7 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 134.3/134.3 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 145.4/145.4 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 199.8/199.8 kB 1.7 MB/s eta 0:00:00


In [2]:
import openai
import numpy as np
from datasets import load_dataset, list_datasets
import pandas as pd
import numpy as np

openai.api_key = 'enter-your-api-key-here'

c:\Users\radue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Test API key

In [37]:
response = openai.Completion.create(
      engine="text-davinci-002",
      prompt='A transformer is a device that uses electricity to transfer energy from one circuit to another through a process called electrical induction',
      temperature=0.5,
      max_tokens=256, # Recommended >256
      frequency_penalty=0,
      presence_penalty=0,
      best_of=1, # Recommended =3,5
      n=1
  )
response

<OpenAIObject text_completion id=cmpl-734v8iuIcbicXVKoYoKeaHFUM99jP at 0x1d913610d60> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": ". A transformer is a static device, which means it doesn't have any moving parts.\n\nThe primary winding of a transformer is the winding that is connected to the input voltage. The secondary winding is the winding that is connected to the output voltage. The primary and secondary windings are wrapped around a common iron core.\n\nWhen the primary winding is energized with an AC voltage, a magnetic field is set up in the iron core. This magnetic field induces a voltage in the secondary winding. The secondary voltage is proportional to the number of turns in the secondary winding, and is inversely proportional to the number of turns in the primary winding. This relationship is known as the transformer equation.\n\nThe transformer equation is:\n\nVsecondary = (Nsecondary / Nprimary) * Vp

## Make ChatGPT Insertions into Human Text

In [2]:
def insert_machine_text(text: str, max_proportion: int):
  text = text.strip()

  n_chars = len(text)
  if n_chars == 0:
    raise Exception("Received empty string '' text")
  
  action = np.random.choice(['prepend', 'insert', 'append'], size=1)
  prefix_text = None
  suffix_text = None
  if action == 'prepend':
    prefix_text = text
  elif action == 'append':
    suffix_text = text
  else:
    # Find white spaces in given text
    white_space_idxs = [i for i, char in enumerate(text) if char == " "]
    # Randomly choose where to insert text
    split_idx = np.random.choice(white_space_idxs, size=1)
    prefix_text = text[:split_idx]
    suffix_text = text[split_idx:]

  max_tokens = int(n_chars * max_proportion)

  # Best practices: https://platform.openai.com/docs/guides/completion/inserting-text
  response = openai.Completion.create(
      engine="text-davinci-002",
      prompt=prefix_text,
      suffix=suffix_text,
      temperature=0.5,
      max_tokens=max_tokens, # Recommended >256
      frequency_penalty=0,
      presence_penalty=0,
      best_of=3, # Recommended =3,5
      n=1
  )
  inserted_text = response.choices[0].text.strip()
  full_text = prefix_text + inserted_text + suffix_text
  return {
      'prefix_text': prefix_text,
      'inserted_text': inserted_text,
      'suffix_text': suffix_text,
      'machine_to_human_ratio': len(inserted_text) / len(prefix_text) + len(suffix_text) 
  }


$f(t) = \sum_i \alpha_i t_i$

Let $f_i$ be the features of the model and $s_i, s_t$ be the text inputs corresponding to feature vectors $x_i$ and $t$, respectively.

$t = (f_1(s_t) \ f_2(s_t) \ \dots \ f_n(s_t))$

Since $t$ is a combination of human text with machine-generated text, we have that $t = (f_1(s_h : s_m) \ f_2(s_h : s_m) \ \dots \ f_n(s_h : s_m))$, where $:$ represents concatenation of strings.

Assume features are proportionally additive and order of concatenation does not affect the value of the functions i.e., $f(s_i : s_j) = \beta_{f,s_i,s_j} [f(s_i) + f(s_j)]$ and $f(s_i : s_j) = f(s_j : s_i)$. For example, for perplexity $\beta_{i,j}$ can be $\frac{}{}$

$t = (f_1(s_h : s_m) \ \dots \ f_n(s_h : s_m))$

$= (\beta_{f_1,h,m} [f_1(h) + f_1(m)] \ \dots \ \beta_{f_n, h, m} [f_n(h) + f_n(m)])$

$ = b_{h,m} \cdot (f(h) + f(m) \ \dots \ f(h) + f(m))$

Therefore,

$f(t) = \sum_i \alpha_i \beta_{f_i,h,m} [f_i(h) + f_i(m)]$
$= \sum_i \alpha_i \beta_{f_i,h,m} f_i(h) + \sum_i \alpha_i \beta_{f_i,h,m} f_i(m)$

If $\beta_{f_i,h,m}$ is independent of $h$ and $m$, then:

$f(t) = b \cdot f(h) + b \cdot f(m) = b \cdot (f(h) + f(m))$ for all $t$ that are combinations of human and machine-generated text.

Thus, $f(t)$ is proportional to


### Make Rephrases of ChatGPT answers

In [3]:
df = pd.read_json('all.jsonl', lines=True)

In [4]:
df['chatgpt_answers'][100][0]

'A transformer is a device that uses electricity to transfer energy from one circuit to another through a process called electrical induction. It does this by using coils of wire and a magnetic field. \nInside a transformer, there are two coils of wire called the primary coil and the secondary coil. The primary coil is connected to a source of electricity, such as a power outlet, and the secondary coil is connected to a device that needs electricity, such as a lamp. \nWhen electricity flows through the primary coil, it creates a magnetic field around the coil. This magnetic field then causes a current to flow in the secondary coil, which creates its own magnetic field. The two magnetic fields interact with each other, causing energy to be transferred from the primary coil to the secondary coil. \nThe amount of energy that is transferred depends on the number of coils in each of the primary and secondary coils. If the secondary coil has fewer coils than the primary coil, it will receive

Find max number of tokens in first 2k points in dataset ([1 token is approx. 4 chars in English](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them)).

In [5]:
n_answers_total = 0
n_tokens_total = 0
tokens_count_list = []
for answers in df['chatgpt_answers']:
  n_answers_total += len(answers)
  n_tokens_max = np.max([len(answer) / 4 for answer in answers])
  n_tokens_total += np.sum([len(answer) / 4 for answer in answers])
  tokens_count_list.append(n_tokens_max)
  
print('Num max tokens per chatgpt_answers column = %.2f' % np.max(tokens_count_list))
print('Avg num tokens per chatgpt_answers column = %.2f' % (n_tokens_total / n_answers_total))

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
n_answers_total = 0
n_tokens_total = 0
tokens_count_list = []
for answers in df['human_answers']:
  n_answers_total += len(answers)
  n_tokens_max = np.max([len(answer) / 4 for answer in answers])
  n_tokens_total += np.sum([len(answer) / 4 for answer in answers])
  tokens_count_list.append(n_tokens_max)

print('Num max tokens per human_answers column = %.2f' % np.max(tokens_count_list))
print('Avg num tokens per human_answers column = %.2f' % (n_tokens_total / n_answers_total))

Num max tokens per human_answers column = 9198.25
Avg num tokens per human_answers column = 170.17


In [ ]:
df.shape

(24322, 5)

Send rephrasing requests to ChatGPT API

In [7]:

# Process them in batches of 250, to avoid API limits
batch_size = 250
for batch_low_idx in range(5250, 5750, batch_size):
  map_rephrased = {
    'index': [],
    'original_chatgpt': [],
    'rephrased_chatgpt': [],
  }
  for i in range(batch_low_idx, batch_low_idx + batch_size):
    original_texts = df['chatgpt_answers'][i]

    if len(original_texts) == 0:
      print('WARNING: Found empty chatgpt original text at index i=%d' % i)
      continue
    
    original_text = original_texts[0]

    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt="Rephrase the following text and make it one paragraph long: '" + original_text + "'",
      temperature=0.9,
      max_tokens=1024, # larger than 637
      frequency_penalty=0,
      presence_penalty=0,
      best_of=1,
      n=1
    )

    # headers: index, original_chatgpt, rephrased_chatgpt
    map_rephrased['index'].append(i)
    map_rephrased['original_chatgpt'].append(original_text)
    map_rephrased['rephrased_chatgpt'].append(response.choices[0].text.strip())

    # Count number of empty response from OpenAI API
    count_empty = 0
    for rephrased_answer in map_rephrased['rephrased_chatgpt']:
      if rephrased_answer == None or rephrased_answer == '' or rephrased_answer == 'NaN':
        count_empty += 1
    print('Got %d empty responses in batch %d-%d' % (count_empty, batch_low_idx, batch_low_idx + batch_size))

    # Write rephrases to csv
    df_rephrased = pd.DataFrame.from_dict(map_rephrased)
    df_rephrased.to_csv('rephrased_first_' + '%d_%d' % (batch_low_idx, batch_low_idx + batch_size) + '.csv')

Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty responses in batch 5250-5500
Got 0 empty resp

In [33]:
count = 0
for rephrased_answer in map_rephrased['rephrased_chatgpt']:
  if rephrased_answer == '':
    count += 1
count

0

In [34]:
df_rephrased = pd.DataFrame.from_dict(map_rephrased)
df_rephrased.to_csv('rephrased_first_3000_3500.csv')

In [35]:
df_test = pd.read_csv('rephrased_first_3000_3500.csv')
df_test

,Unnamed: 0,index,original_chatgpt,rephrased_chatgpt
0,0,2500,Bitcoins are a type of digital currency that c...,Bitcoins can be obtained through either mining...
1,1,2501,Different countries have different electrical ...,Different countries have electrical plugs that...
2,2,2502,The Stop Online Piracy Act (SOPA) was a bill t...,The Stop Online Piracy Act (SOPA) was a bill p...
3,3,2503,Sure! The Standard Model of particle physics i...,The Standard Model of particle physics is a th...
4,4,2504,"Sure! A processor, also known as a Central Pro...","A processor, commonly known as a Central Proce..."
...,...,...,...,...
495,495,2995,Sure! DES and AES are types of encryption that...,DES and AES are two types of encryption used t...
496,496,2996,Great question! When a government wants to pri...,"When a government wants to print more money, t..."
497,497,2997,Outsourcing is when a company hires someone fr...,Outsourcing can be a good or bad thing dependi...
498,498,2998,Strategy and tactics are both ways of achievin...,Strategy and tactics are two levels of action ...


In [36]:
df_test['rephrased_chatgpt'].isna().sum()

0

In [9]:
df_all = pd.concat(
    [
        pd.read_csv('rephrased_first_1k.csv'),
        pd.read_csv('rephrased_first_1000_1300.csv'),
        pd.read_csv('rephrased_first_1300_1600.csv'),
        pd.read_csv('rephrased_first_1600_2000.csv'),
        pd.read_csv('rephrased_first_2000_2500.csv'),
        pd.read_csv('rephrased_first_2500_3000.csv'),
        pd.read_csv('rephrased_first_3000_3250.csv'),
        pd.read_csv('rephrased_first_3250_3500.csv'),
        pd.read_csv('rephrased_first_3500_3750.csv'),
        pd.read_csv('rephrased_first_3750_4000.csv'),
        pd.read_csv('rephrased_first_4000_4250.csv'),
        pd.read_csv('rephrased_first_4250_4500.csv'),
        pd.read_csv('rephrased_first_4500_4750.csv'),
        pd.read_csv('rephrased_first_4750_5000.csv'),
        pd.read_csv('rephrased_first_5000_5250.csv'),
        pd.read_csv('rephrased_first_5250_5500.csv'),
        pd.read_csv('rephrased_first_5500_5750.csv'),
    ],
    ignore_index=True,
    axis=0

)
df_all.shape

(5750, 5)

In [ ]:
df_all.to_csv('rephrased_first_5750.csv')

In [49]:
df_all[df_all['rephrased_chatgpt'].isnull()]

,Unnamed: 0,index,original_chatgpt,rephrased_chatgpt
167,117,167,Tilting your rearview mirror up at night allow...,NaN


In [50]:
df_all.iloc[167]['original_chatgpt']

"Tilting your rearview mirror up at night allows you to see a dimmer version of what's behind you because the mirror is designed to reflect light from the rear of your vehicle. When you tilt the mirror up, the light from the headlights of the vehicles behind you is reflected off the mirror and into your eyes. This can be helpful at night because it allows you to see what's behind you without being blinded by the bright headlights. \nThe mirror is not just tilting up to hit the ceiling inside your car. It is still reflecting light, just at a different angle. When you tilt the mirror up, you are changing the angle at which the light is reflected off the mirror and into your eyes. This change in angle causes the light to appear dimmer, which can make it easier to see what's behind you at night."

In [37]:
original_text = df_all.iloc[167]['original_chatgpt']

response = openai.Completion.create(
engine="text-davinci-003",
prompt="Rephrase the following text and make it one paragraph long: '" + original_text + "'",
temperature=0.9,
max_tokens=1024, # larger than 637
frequency_penalty=0,
presence_penalty=0,
best_of=1,
n=1
)

In [51]:
df_all.loc[167, 'rephrased_chatgpt'] = response.choices[0].text

In [54]:
print(df_all.iloc[167]['original_chatgpt'])
print()
print(df_all.iloc[167]['rephrased_chatgpt'])

Tilting your rearview mirror up at night allows you to see a dimmer version of what's behind you because the mirror is designed to reflect light from the rear of your vehicle. When you tilt the mirror up, the light from the headlights of the vehicles behind you is reflected off the mirror and into your eyes. This can be helpful at night because it allows you to see what's behind you without being blinded by the bright headlights. 
The mirror is not just tilting up to hit the ceiling inside your car. It is still reflecting light, just at a different angle. When you tilt the mirror up, you are changing the angle at which the light is reflected off the mirror and into your eyes. This change in angle causes the light to appear dimmer, which can make it easier to see what's behind you at night.

 Tilting your rearview mirror up at night allows you to see a dimmer version of what's behind you, making it easier to observe without being blinded by bright headlights, as the mirror is designed t

In [59]:
df_all[df_all['rephrased_chatgpt'].isnull()]

,Unnamed: 0,index,original_chatgpt,rephrased_chatgpt


In [56]:
df_all.to_csv('rephrased_first_1k.csv')

In [ ]:
df_all[df_all['column name'].isnull()]

In [ ]:
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt="Rephrase the following text: " + df['chatgpt_answers'][0][0],
    temperature=0.5,
    max_tokens=200,
    frequency_penalty=0,
    presence_penalty=0,
    best_of=1,
    n=1,
  )

In [ ]:
response

<OpenAIObject text_completion id=cmpl-6zOuwxX1GQ0INdAbRo3gGQj55p7lf at 0x7fb464bb13b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nThe New York Times publishes a weekly best seller list which ranks the most popular books in the United States based on sales data from various outlets. This list is widely regarded as one of the most influential in the book industry and is often referred to when discussing best sellers. However, it is not the only best seller list available and there are many other lists which rank the top-selling books in various categories or countries. Therefore, a book can be a best seller on one list, but not on another. Additionally, the term \"best seller\" is often used more generally to refer to any book that is selling well, regardless of whether it is featured on a particular list or not."
    }
  ],
  "created": 1680091810,
  "id": "cmpl-6zOuwxX1GQ0INdAbRo3gGQj55p7lf",
  "model": "text-d

In [ ]:
map_rephrased

{'index': [0, 1, 2],
 'original_chatgpt': ['There are many different best seller lists that are published by various organizations, and the New York Times is just one of them. The New York Times best seller list is a weekly list that ranks the best-selling books in the United States based on sales data from a number of different retailers. The list is published in the New York Times newspaper and is widely considered to be one of the most influential best seller lists in the book industry. \nIt\'s important to note that the New York Times best seller list is not the only best seller list out there, and there are many other lists that rank the top-selling books in different categories or in different countries. So it\'s possible that a book could be a best seller on one list but not on another. \nAdditionally, the term "best seller" is often used more broadly to refer to any book that is selling well, regardless of whether it is on a specific best seller list or not. So it\'s possible t

Try with Edits API

In [21]:
text = df['chatgpt_answers'][100][0]
text

'A transformer is a device that uses electricity to transfer energy from one circuit to another through a process called electrical induction. It does this by using coils of wire and a magnetic field. \nInside a transformer, there are two coils of wire called the primary coil and the secondary coil. The primary coil is connected to a source of electricity, such as a power outlet, and the secondary coil is connected to a device that needs electricity, such as a lamp. \nWhen electricity flows through the primary coil, it creates a magnetic field around the coil. This magnetic field then causes a current to flow in the secondary coil, which creates its own magnetic field. The two magnetic fields interact with each other, causing energy to be transferred from the primary coil to the secondary coil. \nThe amount of energy that is transferred depends on the number of coils in each of the primary and secondary coils. If the secondary coil has fewer coils than the primary coil, it will receive

In [28]:
response = response = openai.Edit.create(
      model="text-davinci-edit-001",
      input=text,
      instruction="Rephrase the given text and make it one paragraph long",
      temperature=0.5,
      n=4, # num edits to generate, can play with this
  )

In [27]:
response.choices[0].text

'A transformer is a device that uses electricity to transfer energy from one circuit to another through a process called electrical induction. It does this by using coils of wire and a magnetic field. Inside a transformer, there are two coils of wire called the primary coil and the secondary coil. The primary coil is connected to a source of electricity, such as a power outlet, and the secondary coil is connected to a device that needs electricity, such as a lamp. When electricity flows through the primary coil, it creates a magnetic field around the coil. This magnetic field then causes a current to flow in the secondary coil, which creates its own magnetic field. The two magnetic fields interact with each other, causing energy to be transferred from the primary coil to the secondary coil. The amount of energy that is transferred depends on the number of coils in each of the primary and secondary coils. If the secondary coil has fewer coils than the primary coil, it will receive less 

Is always very redundant to making significant changes to the original text (note it is currently in alpha)

Try with Completions API

In [31]:
text = df['chatgpt_answers'][100][0]
text

'A transformer is a device that uses electricity to transfer energy from one circuit to another through a process called electrical induction. It does this by using coils of wire and a magnetic field. \nInside a transformer, there are two coils of wire called the primary coil and the secondary coil. The primary coil is connected to a source of electricity, such as a power outlet, and the secondary coil is connected to a device that needs electricity, such as a lamp. \nWhen electricity flows through the primary coil, it creates a magnetic field around the coil. This magnetic field then causes a current to flow in the secondary coil, which creates its own magnetic field. The two magnetic fields interact with each other, causing energy to be transferred from the primary coil to the secondary coil. \nThe amount of energy that is transferred depends on the number of coils in each of the primary and secondary coils. If the secondary coil has fewer coils than the primary coil, it will receive

In [42]:
"Rephrase the following text and make it one paragraph long: '" + text + "'"

"Rephrase the following text and make it one paragraph long: 'A transformer is a device that uses electricity to transfer energy from one circuit to another through a process called electrical induction. It does this by using coils of wire and a magnetic field. \nInside a transformer, there are two coils of wire called the primary coil and the secondary coil. The primary coil is connected to a source of electricity, such as a power outlet, and the secondary coil is connected to a device that needs electricity, such as a lamp. \nWhen electricity flows through the primary coil, it creates a magnetic field around the coil. This magnetic field then causes a current to flow in the secondary coil, which creates its own magnetic field. The two magnetic fields interact with each other, causing energy to be transferred from the primary coil to the secondary coil. \nThe amount of energy that is transferred depends on the number of coils in each of the primary and secondary coils. If the secondar

In [52]:
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt="Rephrase the following text and make it one paragraph long: '" + text + "'",
    temperature=0.9,
    max_tokens=1024,
    frequency_penalty=0,
    presence_penalty=0,
    n=1,
    best_of=1
)

In [53]:
response.choices[0].text

'\n\nA transformer uses electricity to transfer energy from one circuit to another through a process called electrical induction by using coils of wire and a magnetic field. It contains two coils of wire, the primary coil connected to a source of electricity, such as a power outlet, and the secondary coil connected to a device needing electricity, such as a lamp. When electricity flows through the primary coil, it creates a magnetic field that induces a current in the secondary coil, thus transferring energy from the primary coil to the secondary coil. The amount of energy transferred depends on the number of coils in each of the primary and secondary coils - more coils in the secondary coil leads to more energy and higher voltage in the secondary circuit, and vice versa. Does this help to explain how a transformer works?'

### Rephrase First 1k Human Answers

In [8]:
df

,question,human_answers,chatgpt_answers,index,source
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,reddit_eli5
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,reddit_eli5
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,reddit_eli5
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,reddit_eli5
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,reddit_eli5
...,...,...,...,...,...
1995,Why is it that countries with a smaller differ...,[Right now we live in what is referred to as a...,[A large gap between rich and poor can be bad ...,388.0,reddit_eli5
1996,Tone deafness : what is it exactly and how in ...,[It 's when someone ca n't tell the difference...,"[Tone deafness, also known as amusia, is a con...",389.0,reddit_eli5
1997,"First grade math Prove to me that 2 + 2=4 , an...","[This is a good question , one that many mathe...",[Sure! 2 + 2 is equal to 4 because if you have...,390.0,reddit_eli5
1998,What exactly are tropes ? And more specificall...,[A trope is just a commonly expressed pattern ...,"[Tropes are a common pattern, theme, or motif ...",391.0,reddit_eli5


In [17]:
map_rephrased = {
    'index': [],
    'original_human': [],
    'rephrased_human': [],
}
# 700-1000
for i in range(700, 1000):
  original_texts = df['human_answers'][i]

  if len(original_texts) == 0:
    print('WARNING: Found empty human original text at index i=%d' % i)
    continue
  
  original_text = original_texts[0]

  response = openai.Completion.create(
    engine="text-davinci-003",
    prompt="Rephrase the following text and make it one paragraph long: '" + original_text + "'",
    temperature=0.9,
    max_tokens=1024, # larger than 637
    frequency_penalty=0,
    presence_penalty=0,
    best_of=1,
    n=1
  )

  # headers: index, original_chatgpt, rephrased_chatgpt
  map_rephrased['index'].append(i)
  map_rephrased['original_human'].append(original_text)
  map_rephrased['rephrased_human'].append(response.choices[0].text.strip())

In [18]:
count = 0
for rephrased_answer in map_rephrased['rephrased_human']:
  if rephrased_answer == '':
    count += 1
count

0

In [19]:
df_rephrased = pd.DataFrame.from_dict(map_rephrased)
df_rephrased.to_csv('human_rephrased_first_700_1000.csv')

In [20]:
df_test = pd.read_csv('human_rephrased_first_700_1000.csv')
df_test

,Unnamed: 0,index,original_human,rephrased_human
0,0,700,I think the real question is how you define bi...,What constitutes biological success is up for ...
1,1,701,The music industry has little to do with music...,The music industry is more about business than...
2,2,702,"Well , in terms of actual BDSM it is actually ...","Well, in terms of actual BDSM, Fifty Shades of..."
3,3,703,Glucose monitors have been around for over 35 ...,"For over 35 years, glucose monitors have been ..."
4,4,704,If you want to look at it from a monetary pers...,"Exploring space not only creates new tech, exp..."
...,...,...,...,...
295,295,995,The data is on your hard drive and when you de...,"If you delete data stored on your hard drive, ..."
296,296,996,"Okay , so I think everyone is different . For ...",When I'm nervous or stressed I tend to experie...
297,297,997,Expense and privacy concerns . If a police off...,Given the expense and privacy issues that aris...
298,298,998,"[ Here 's Kevin James ] ( URL_2 ) , and [ here...",Kevin James (URL_2) and his love interest in P...


In [21]:
df_test['rephrased_human'].isna().sum()

0

In [24]:
df_1 = pd.read_csv('human_rephrased_first_01_100.csv')
df_2 = pd.read_csv('human_rephrased_first_100_400.csv')
df_3 = pd.read_csv('human_rephrased_first_400_700.csv')
df_4 = pd.read_csv('human_rephrased_first_700_1000.csv')

df_all = pd.concat([df_1, df_2, df_3, df_4], ignore_index=True, axis=0)
df_all

,Unnamed: 0,index,original_human,rephrased_human
0,0,0,"Basically there are many categories of "" Best ...","Basically, similar to the Oscars, if a book is..."
1,1,1,salt is good for not dying in car crashes and ...,Salt is an inexpensive and effective way to pr...
2,2,2,The way it works is that old TV stations got a...,Old TV stations were given a certain amount of...
3,3,3,You ca n't just go around assassinating the le...,Even though no one likes Kim Jong-Un and North...
4,4,4,Wanting to kill the shit out of Germans drives...,The motivating desire to defeat the Germans in...
...,...,...,...,...
995,295,995,The data is on your hard drive and when you de...,"If you delete data stored on your hard drive, ..."
996,296,996,"Okay , so I think everyone is different . For ...",When I'm nervous or stressed I tend to experie...
997,297,997,Expense and privacy concerns . If a police off...,Given the expense and privacy issues that aris...
998,298,998,"[ Here 's Kevin James ] ( URL_2 ) , and [ here...",Kevin James (URL_2) and his love interest in P...


In [25]:
df_all['rephrased_human'].isna().sum()

0

In [26]:
df_all.to_csv('human_rephrased_first_1000.csv')